In [2]:
from z3 import *

In [ ]:
row_0 = Int('row_0')
col_0 = Int('col_0')

In [ ]:
def get_restrictions_select(in_row, in_col, out_row, out_col):
    r1 = in_row==out_row
    r2 = in_col>=out_col
    return (r1, r2)

In [80]:
def abs_select(in_row, in_col, out_row, out_col):
    r_row = And(out_row==in_row,in_row>0,out_row>0)
    r_col = And(Or(out_col==1,out_col==2),in_col>0,out_col>0)
    return (r_row, r_col)

def abs_unite(in_row, in_col, out_row, out_col):
    r_row = And(out_row==in_row,in_row>0,out_row>0)
    r_col = And(out_col==in_col-1,in_col>0,out_col>0)
    return (r_row, r_col)

def abs_gather(in_row, in_col, out_row, out_col):
    r_row = And(out_row>=in_row,in_row>0,out_row>0)
    r_col = And(Or(out_col==in_col,out_col==in_col-1),in_col>0,out_col>0)
    return (r_row, r_col)

def abs_spread(in_row, in_col, out_row, out_col):
    r_row = And(out_row<=in_row,in_row>0,out_row>0)
    r_col = And(And(out_col>=in_col,out_col<=in_col+in_row),in_col>0,out_col>0)
    return (r_row, r_col)

def abs_separate(in_row, in_col, out_row, out_col):
    r_row = And(out_row==in_row,in_row>0,out_row>0)
    r_col = And(out_col==in_col+1,in_col>0,out_col>0)
    return (r_row, r_col)

In [81]:
abs_funcs = {
    "select": abs_select,
    "unite": abs_unite,
    "gather": abs_gather,
    "spread": abs_spread,
    "separate": abs_separate,
}

In [82]:
def construct_pr(n_steps, p_input, p_output):
    xlist = []
    ylist = []
    problems = []
    rlist = []
    for i in range(n_steps+1):
        tmpx = Int("x_{}".format(i))
        tmpy = Int("y_{}".format(i))
        xlist.append(tmpx)
        ylist.append(tmpy)
        if i==0:
            # input, continue
            continue
        else:
            zlist = []
            for dkey in abs_funcs.keys():
                rx,ry = abs_funcs[dkey](xlist[i-1],ylist[i-1],tmpx,tmpy)
                rz = And(rx,ry)
                zlist.append(rz)
            rlist.append(Or(zlist))
    restrictions = [And(rlist)]
    problems.append(xlist[0]==p_input[0])
    problems.append(ylist[0]==p_input[1])
    problems.append(xlist[-1]==p_output[0])
    problems.append(ylist[-1]==p_output[1])
    # solve(problems+restrictions)
    return problems, restrictions

In [83]:
def construct_pr_assumption(n_steps, p_input, p_output, p_choose):
    xlist = []
    ylist = []
    problems = []
    rlist = []
    for i in range(n_steps+1):
        tmpx = Int("x_{}".format(i))
        tmpy = Int("y_{}".format(i))
        xlist.append(tmpx)
        ylist.append(tmpy)
        if i==0:
            # input, continue
            continue
        else:
            zlist = []
            if i==1:
                # force to choose a key
                rx,ry = abs_funcs[p_choose](xlist[i-1],ylist[i-1],tmpx,tmpy)
                rz = And(rx,ry)
                zlist.append(rz)
            else:
                for dkey in abs_funcs.keys():
                    rx,ry = abs_funcs[dkey](xlist[i-1],ylist[i-1],tmpx,tmpy)
                    rz = And(rx,ry)
                    zlist.append(rz)
            rlist.append(Or(zlist))
    restrictions = [And(rlist)]
    problems.append(xlist[0]==p_input[0])
    problems.append(ylist[0]==p_input[1])
    problems.append(xlist[-1]==p_output[0])
    problems.append(ylist[-1]==p_output[1])
    # solve(problems+restrictions)
    return problems, restrictions

In [63]:
dp, dr = construct_pr(1, (6,4), (20,5))

In [86]:
dp, dr = construct_pr_assumption(2, (6,10), (6,9), "unite")

In [87]:
solve(dp+dr)

[y_1 = 9, x_1 = 6, y_2 = 9, x_2 = 6, y_0 = 10, x_0 = 6]


In [26]:
x_in, y_in, x_out, y_out = Ints('x_in y_in x_out y_out')

a,b = abs_select(x_in,y_in,x_out,y_out)
c,d = abs_unite(x_in,y_in,x_out,y_out)
z = Or(And(a,b),And(c,d))

problems = [
    x_in == 6,
    y_in == 4,
    x_out== 6,
    y_out== 2,
]
restrictions = [
    z
]


In [27]:
solve(problems+restrictions)

[y_out = 2, x_out = 6, y_in = 4, x_in = 6]
